In [ ]:
#When running on Jupyter Lab
import sys
!{sys.executable} -m pip install biopython
import random
import pandas as pd
from Bio.Seq import Seq
from nupack import *

nuc_list=['A', 'T', 'G', 'C']

#Illegal sequences, based on restriction enzymes
ill_seq=['GAATTC', 'TCTAGA', 'ACTAGT', 'CTGCAG', 'GCGGCCGC']

#Checks for invalid sequences of input
def check(x):
  for i in range(len(x)):
    if x[i]!='U' and x[i]!='T' and x[i]!='A' and x[i]!='G' and x[i]!='C':
      y='Invalid sequence, run the program again'
      break
    else:
      y=''
  return y

#From Nupack
def gibbs(Y1,Y2,Y3):
	
	Y1=Strand(str(Y1), name='Y1')
	Y2=Strand(str(Y2), name='Y2')
	Y3=Strand(str(Y3), name='Y3')
	set1 = ComplexSet(strands=[Y1, Y2, Y3], complexes=SetSpec(max_size=3))

	my_model= Model(material='dna', ensemble='stacking', celsius=37)
	complex_results = complex_analysis(complexes=set1, model=my_model, compute=['pfunc'])
	
	rstr=str(complex_results)
	
	l_rows=rstr.split('\n')

	chars = ['Y1','Y2','Y3']
	f = []
	
	for j in range(len(l_rows)):
		
		has_all = all([char in l_rows[j] for char in chars])
		
		
		if has_all == True:
			f.append(l_rows[j])
			
	gibbs = []		
	for i in f:
		gibbs.append(i.split('-')[-1])
		
	final = max(gibbs)	
	return final

#Setting the mir viariables
mir1=input("Give miR-1 sequence:\n")
mir1=mir1.replace(" ","")
mir1.upper()
mir1=Seq(mir1)
mir2=input("Give miR-2 sequence:\n")
mir2=mir2.replace(" ","")
mir2.upper()
mir2=Seq(mir2)

#mir1=Seq('UAGCUUAUCAGACUGAUGUUGA')
#mir2=Seq('UACCCUGUAGAACCGAAUUUGUG')

y1=check(mir1)
print(y1)
y2=check(mir2)
print(y2)

#Setting the complementary variables
cmir1=mir1.complement().back_transcribe()
cmir2=mir2.complement().back_transcribe()

#Asking for the rest of the variables
#tl1=int(input('Give length of toehold for the first miRNA'))
#tl2=int(input('Give length of toehold for the second miRNA'))
un=3
tl1=8
tl2=6

#Setting the standard sequences

#Y2 is complementary to miR-2 and reversed
Y2=cmir2[::-1]

#First bases of Y1 are complementary to the last bases of Y2 and reversed (the length is that of the second toehold)
Y2_rev=Y2[::-1]
Y1_a=Y2_rev.complement()[0:tl2]

#The rest is complementary to miR-1 and reversed
Y1_c=cmir1[::-1]

#The Y3 molecule acts as the initiator and depends on the miR sequences
Y1_crev=Y1_c[::-1]
Y3_a=Y1_crev.complement()[tl1::]
Y3_b=Y2_rev.complement()[tl2:]
Y3=Y3_a+Y3_b

for i in range(len(ill_seq)):
	if ill_seq[i] in Y2:
		print('Illegal sequence in Y2, check your inputs')
		print(Y3)
		sys.exit()
	if ill_seq[i] in Y3:
		print('Illegal sequence in Y3, check your inputs')
		print(Y3)
		sys.exit()

NUM_ITS=100

df=pd.DataFrame(columns=['ID','Y1','Y2','Y3','Gibbs'])

id=df['ID']
max_g = str(0)

for step in range(NUM_ITS):
	x=0

	Y1_blist=[]
	for i in range(un):
		Y1_blist.append(random.choice(nuc_list))
		Y1_b=Seq(''.join(Y1_blist))
		Y1=Y1_a+Y1_b+Y1_c

	for i in range(len(ill_seq)):
		if ill_seq[i] in Y1:
			x=1
		if x==1:
			continue
     
	g = gibbs(Y1,Y2,Y3)
	if g>max_g:
		max_g =g
		max_i = step
df.at[step,'ID']=step
df.at[step,'Y1']=str(Y1)
df.at[step,'Y2']=str(Y2)
df.at[step,'Y3']=str(Y3)
df.at[step,'Gibbs']='-' + str(g)

print(df.iloc[max_i])

df.to_csv('Y sequences.csv')

df